In [1]:
import numpy as np
import dash
from dash import dcc, html, Dash
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [2]:
import pickle

# Carregar o modelo NearestNeighbors
with open('artifacts/model.pkl', 'rb') as file:
    model = pickle.load(file)

# Carregar os nomes dos livros
with open('artifacts/books_name.pkl', 'rb') as file:
    books_name = pickle.load(file)

# Carregar os dados de avaliações finais
with open('artifacts/final_rating.pkl', 'rb') as file:
    final_rating = pickle.load(file)

# Carregar a matriz pivot
with open('artifacts/book_pivot.pkl', 'rb') as file:
    book_pivot = pickle.load(file)

In [3]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index==book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
    
    recommendations = []

    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            url = final_rating.loc[final_rating['title'].str.contains(j, case=False, regex=False), 'image_url'].values[0]
            recommendation = {'Book': j, 'URL Image': url}
            recommendations.append(recommendation)

    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

In [4]:
#book_name = 'Harry Potter and the Chamber of Secrets (Book 2)'
#recommend_book(book_name)

In [5]:
#recommend_book(book_name)['URL Image'][5]

In [6]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

app.layout = dbc.Container([

    dbc.Row([
        dbc.Col(html.H1('Book Recommender System', className = 'text-center text-primary mb-4'), width = 12)
    ]),
    dbc.Row([
        dbc.Col(html.H5('Select a book to see the recommendations:'), width = 12)
    ]),
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id = 'book-name',
            options = [{'label': i, 'value': i} for i in books_name],
            value = "Fear Nothing"
        ), width = 12)
    ]),
    dbc.Row([
        dbc.Col(html.Hr(), width = 12)
    ]),
    dbc.Row([
        dbc.Col(html.H5('Recommendations:'), width = 12)
    ]),
    dbc.Row([
        html.Div(id = 'output')
    ])

], fluid = True)

@app.callback(
    Output('output', 'children'),
    Input('book-name', 'value')
)

def update_output_div(input_value):
    data = recommend_book(input_value)
    estrutura = [
        dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src = data['URL Image'][0], top = True, id = 'image1'),
                dbc.CardBody([
                    html.H5(children=data['Book'][0], className = 'card-title', id = 'title1'),
                ])
            ], style = {'width': '18rem'})
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src = data['URL Image'][1], top = True, id = 'image2'),
                dbc.CardBody([
                    html.H5(children=data['Book'][1], className = 'card-title', id = 'title2'),
                ])
            ], style = {'width': '18rem'})
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src = data['URL Image'][2], top = True, id = 'image3'),
                dbc.CardBody([
                    html.H5(children=data['Book'][2], className = 'card-title', id = 'title3'),
                ])
            ], style = {'width': '18rem'})
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src = data['URL Image'][3], top = True, id = 'image4'),
                dbc.CardBody([
                    html.H5(children=data['Book'][3], className = 'card-title', id = 'title4'),
                ])
            ], style = {'width': '18rem'})
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src = data['URL Image'][4], top = True, id = 'image5'),
                dbc.CardBody([
                    html.H5(children=data['Book'][4], className = 'card-title', id = 'title5'),
                ])
            ], style = {'width': '18rem'})
        ]),
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src = data['URL Image'][5], top = True, id = 'image6'),
                dbc.CardBody([
                    html.H5(children=data['Book'][5], className = 'card-title', id = 'title6'),
                ])
            ], style = {'width': '18rem'})
        ])
    ])
    ]

    return estrutura
    
if __name__ == "__main__":
    app.run_server()
   # app.run(jupyter_mode="external")
